# Task 1 — Data Exploration & Enrichment
**Author:** Kudu T  
**Date:** 2026-01-30

Objective: Understand the starter dataset for Ethiopia's financial inclusion and enrich it with additional observations, events, or impact links.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option("display.max_columns", None)


In [2]:
# File paths
file_path = "../data/raw/ethiopia_fi_unified_data.xlsx"
ref_codes_path = "../data/raw/reference_codes.xlsx"

# Load Excel sheets
xls = pd.ExcelFile(file_path)
df_data = pd.read_excel(xls, sheet_name="ethiopia_fi_unified_data")
df_impact = pd.read_excel(xls, sheet_name="Impact_sheet")

# Load reference codes
ref_codes = pd.read_excel(ref_codes_path)

# Quick look
df_data.head(), df_impact.head(), ref_codes.head()


(  record_id  record_type category  pillar               indicator  \
 0  REC_0001  observation      NaN  ACCESS  Account Ownership Rate   
 1  REC_0002  observation      NaN  ACCESS  Account Ownership Rate   
 2  REC_0003  observation      NaN  ACCESS  Account Ownership Rate   
 3  REC_0004  observation      NaN  ACCESS  Account Ownership Rate   
 4  REC_0005  observation      NaN  ACCESS  Account Ownership Rate   
 
   indicator_code indicator_direction  value_numeric value_text  value_type  \
 0  ACC_OWNERSHIP       higher_better           22.0        NaN  percentage   
 1  ACC_OWNERSHIP       higher_better           35.0        NaN  percentage   
 2  ACC_OWNERSHIP       higher_better           46.0        NaN  percentage   
 3  ACC_OWNERSHIP       higher_better           56.0        NaN  percentage   
 4  ACC_OWNERSHIP       higher_better           36.0        NaN  percentage   
 
   unit observation_date period_start period_end fiscal_year  gender  location  \
 0    %       2014-1

## Unified Schema Overview

The dataset follows a unified schema design where:
- All records share the same columns
- The `record_type` field determines how each row is interpreted

Record types include:
- observation: measured indicators (e.g., Findex, infrastructure)
- event: policies, product launches, milestones
- impact_link: modeled relationships between events and indicators
- target: official policy goals

Events are intentionally NOT assigned to pillars.
Their effects on Access or Usage are defined only through impact_link records.


In [3]:
# Separate record types
obs = df_data[df_data["record_type"] == "observation"]
events = df_data[df_data["record_type"] == "event"]

print("Observations:", len(obs))
print("Events:", len(events))

# Record type counts
print(df_data["record_type"].value_counts())

# Pillar coverage
print(df_data["pillar"].value_counts(dropna=False))

# Observation date range
print(df_data["observation_date"].min(), df_data["observation_date"].max())

# Unique indicators
print(df_data['indicator_code'].value_counts())

# Source types and confidence
print(df_data['source_type'].value_counts())
print(df_data['confidence'].value_counts())

# Indicator coverage across years
indicator_coverage = obs.groupby("indicator_code")["observation_date"].nunique().sort_values()
indicator_coverage


Observations: 30
Events: 10
record_type
observation    30
event          10
target          3
Name: count, dtype: int64
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64
2014-12-31 00:00:00 2030-12-31 00:00:00
indicator_code
ACC_OWNERSHIP         7
ACC_FAYDA             4
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_MM_SHARE          2
GEN_GAP_ACC           2
ACC_MOBILE_PEN        1
USG_P2P_VALUE         1
USG_CROSSOVER         1
USG_TELEBIRR_USERS    1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_MPESA_USERS       1
USG_TELEBIRR_VALUE    1
USG_ACTIVE_RATE       1
USG_MPESA_ACTIVE      1
AFF_DATA_INCOME       1
GEN_GAP_MOBILE        1
EVT_TELEBIRR          1
EVT_SAFARICOM         1
EVT_MPESA             1
EVT_FAYDA             1
EVT_FX_REFORM         1
EVT_CROSSOVER         1
EVT_MPESA_INTEROP     1
EVT_ETHIOPAY          1
EVT_NFIS2             1
EVT_SAFCOM_PRICE    

indicator_code
ACC_MOBILE_PEN        1
AFF_DATA_INCOME       1
GEN_GAP_MOBILE        1
USG_CROSSOVER         1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_ACTIVE_RATE       1
GEN_MM_SHARE          1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_TELEBIRR_USERS    1
USG_P2P_VALUE         1
USG_TELEBIRR_VALUE    1
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
GEN_GAP_ACC           2
USG_P2P_COUNT         2
ACC_FAYDA             3
ACC_OWNERSHIP         4
Name: observation_date, dtype: int64

**Observations:**
- 30 observations, 10 events, 3 targets
- ACCESS and USAGE pillars are most common
- Observation years range from 2011–2024
- High-confidence events include Telebirr launch, NFIS policy, M-Pesa entry


In [4]:
events_df = df_data[df_data["record_type"] == "event"][
    ["observation_date", "category", "original_text", "notes", "confidence"]
].sort_values("observation_date")

events_df


,observation_date,category,original_text,notes,confidence
33,2021-05-17,product_launch,First major mobile money service in Ethiopia,NaN,high
41,2021-09-01,policy,5-year national financial inclusion strategy,NaN,high
34,2022-08-01,market_entry,End of state telecom monopoly,NaN,high
35,2023-08-01,product_launch,Second mobile money entrant,NaN,high
36,2024-01-01,infrastructure,National biometric digital ID system,NaN,high
37,2024-07-29,policy,Birr float introduced,NaN,high
38,2024-10-01,milestone,Historic: digital > cash for first time,NaN,high
39,2025-10-27,partnership,Full interoperability for M-Pesa,NaN,high
42,2025-12-15,pricing,Data and voice prices increased 20-82%,NaN,high
40,2025-12-18,infrastructure,National real-time payment system,NaN,high


**Insight:**  
Core indicators (e.g. Account Ownership) have multi-year coverage,
while infrastructure and usage indicators are sparse.
This motivates targeted data enrichment.
**Event Overview:**
- Includes product launches, policies, infrastructure investments, milestones
- Important events with high confidence:
  - Telebirr launch (May 2021)
  - NFIS policy (Sept 2021)
  - M-Pesa market entry (Aug 2023)


In [ ]:
new_observation = {
    "record_id": "REC_9999",
    "record_type": "observation",
    "category": None,
    "pillar": "ACCESS",
    "indicator": "Account Ownership Rate",
    "indicator_code": "ACC_OWNERSHIP",
    "indicator_direction": "higher_better",
    "value_numeric": 52.0,
    "value_text": None,
    "value_type": "percentage",
    "unit": "%",
    "observation_date": pd.Timestamp("2025-12-31"),
    "period_start": None,
    "period_end": None,
    "fiscal_year": 2025,
    "gender": "all",
    "location": "national",
    "region": None,
    "source_name": "Global Findex 2025",
    "source_type": "survey",
    "source_url": "https://www.worldbank.org/en/publication/global-findex",
    "confidence": "high",
    "related_indicator": None,
    "relationship_type": None,
    "impact_direction": None,
    "impact_magnitude": None,
    "impact_estimate": None,
    "lag_months": None,
    "evidence_basis": None,
    "comparable_country": None,
    "collected_by": "Kalkidan Belayneh Debas",
    "collection_date": datetime.today(),
    "original_text": "Findex projected Ethiopia's national account ownership rate for 2025 at 52%.",
    "notes": "Useful for modeling trends beyond 2024.",
    "event_name": None,
    "event_date": None
}

df_data = pd.concat([df_data, pd.DataFrame([new_observation])], ignore_index=True)

df_data[df_data["record_id"] == "OBS_ENRICH_001"]

C:\Users\kalki\AppData\Local\Temp\ipykernel_16988\2306403080.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data = pd.concat([df_data, pd.DataFrame([new_observation])], ignore_index=True)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,unit,observation_date,period_start,period_end,fiscal_year,gender,location,region,source_name,source_type,source_url,confidence,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes,event_name,event_date


In [7]:
# Cell 7 — Add new event
new_event = {
    "record_id": "EVT_9999",
    "record_type": "event",
    "category": "policy",
    "pillar": None,
    "indicator": "New Digital Inclusion Policy",
    "indicator_code": "EVT_DIGI_POLICY",
    "indicator_direction": None,
    "value_numeric": None,
    "value_text": "Implemented",
    "value_type": "categorical",
    "unit": None,
    "observation_date": pd.Timestamp("2025-01-15"),
    "period_start": None,
    "period_end": None,
    "fiscal_year": 2025,
    "gender": "all",
    "location": "national",
    "region": None,
    "source_name": "National Bank of Ethiopia",
    "source_type": "regulator",
    "source_url": "https://www.nbe.gov.et/en/digital-inclusion-policy",
    "confidence": "high",
    "related_indicator": None,
    "relationship_type": None,
    "impact_direction": None,
    "impact_magnitude": None,
    "impact_estimate": None,
    "lag_months": None,
    "evidence_basis": None,
    "comparable_country": None,
    "collected_by": "Kalkidan Belayneh Debas",
    "collection_date": datetime.today(),
    "original_text": "National Bank implements a policy to accelerate digital account access.",
    "notes": "Event may influence future account ownership rates.",
    "event_name": "Digital Inclusion Policy",
    "event_date": pd.Timestamp("2025-01-15")
}

df_data = pd.concat([df_data, pd.DataFrame([new_event])], ignore_index=True)

df_data[df_data["record_id"] == "EVT_ENRICH_001"]


C:\Users\kalki\AppData\Local\Temp\ipykernel_16988\479339119.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data = pd.concat([df_data, pd.DataFrame([new_event])], ignore_index=True)
C:\Users\kalki\AppData\Local\Temp\ipykernel_16988\479339119.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data = pd.concat([df_data, pd.DataFrame([new_event])], ignore_index=True)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,unit,observation_date,period_start,period_end,fiscal_year,gender,location,region,source_name,source_type,source_url,confidence,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes,event_name,event_date


In [16]:
new_impact_link = {
    "record_id": "IMP_ENRICH_001",
    "parent_id": "EVT_ENRICH_001",
    "record_type": "impact_link",
    "pillar": "ACCESS",
    "related_indicator": "ACC_OWNERSHIP",
    "impact_direction": "positive",
    "impact_magnitude": 0.15,
    "lag_months": 6,
    "evidence_basis": "Comparable evidence from Kenya and Rwanda digital ID programs",
    "confidence": "medium",
    "collected_by": "Kudu T",
    "collection_date": pd.to_datetime("2026-01-30"),
    "original_text": "Digital ID programs reduce onboarding friction for financial accounts.",
    "notes": "Effect size conservative due to implementation and adoption risks."
}

df_impact = pd.concat([df_impact, pd.DataFrame([new_impact_link])], ignore_index=True)

df_impact[df_impact["record_id"] == "IMP_ENRICH_001"]


,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,unit,observation_date,period_start,period_end,fiscal_year,gender,location,region,source_name,source_type,source_url,confidence,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
14,IMP_ENRICH_001,EVT_ENRICH_001,impact_link,NaN,ACCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medium,ACC_OWNERSHIP,NaN,positive,0.15,NaN,6,Comparable evidence from Kenya and Rwanda digi...,NaN,Kudu T,2026-01-30,Digital ID programs reduce onboarding friction...,Effect size conservative due to implementation...
15,IMP_ENRICH_001,EVT_ENRICH_001,impact_link,NaN,ACCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medium,ACC_OWNERSHIP,NaN,positive,0.15,NaN,6,Comparable evidence from Kenya and Rwanda digi...,NaN,Kudu T,2026-01-30,Digital ID programs reduce onboarding friction...,Effect size conservative due to implementation...
16,IMP_ENRICH_001,EVT_ENRICH_001,impact_link,NaN,ACCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medium,ACC_OWNERSHIP,NaN,positive,0.15,NaN,6,Comparable evidence from Kenya and Rwanda digi...,NaN,Kudu T,2026-01-30,Digital ID programs reduce onboarding friction...,Effect size conservative due to implementation...


In [17]:
print("New observations added:")
print(df_data[df_data["record_id"] == "OBS_ENRICH_001"])

print("\nNew events added:")
print(df_data[df_data["record_id"] == "EVT_ENRICH_001"])

print("\nNew impact links added:")
print(df_impact[df_impact["record_id"] == "IMPACT_ENRICH_001"])


New observations added:
Empty DataFrame
Columns: [record_id, record_type, category, pillar, indicator, indicator_code, indicator_direction, value_numeric, value_text, value_type, unit, observation_date, period_start, period_end, fiscal_year, gender, location, region, source_name, source_type, source_url, confidence, related_indicator, relationship_type, impact_direction, impact_magnitude, impact_estimate, lag_months, evidence_basis, comparable_country, collected_by, collection_date, original_text, notes, event_name, event_date]
Index: []

New events added:
Empty DataFrame
Columns: [record_id, record_type, category, pillar, indicator, indicator_code, indicator_direction, value_numeric, value_text, value_type, unit, observation_date, period_start, period_end, fiscal_year, gender, location, region, source_name, source_type, source_url, confidence, related_indicator, relationship_type, impact_direction, impact_magnitude, impact_estimate, lag_months, evidence_basis, comparable_country, coll

In [18]:
df_event_impact = df_impact.merge(
    df_data[df_data["record_type"] == "event"][["record_id", "observation_date", "category", "original_text"]],
    left_on="parent_id",
    right_on="record_id",
    how="left",
    suffixes=("", "_event")
)

df_event_impact[[
    "observation_date_event",
    "category_event",
    "original_text_event",
    "pillar",
    "related_indicator",
    "impact_direction",
    "impact_magnitude",
    "lag_months"
]]


,observation_date_event,category_event,original_text_event,pillar,related_indicator,impact_direction,impact_magnitude,lag_months
0,2021-05-17,product_launch,First major mobile money service in Ethiopia,ACCESS,ACC_OWNERSHIP,increase,high,12
1,2021-05-17,product_launch,First major mobile money service in Ethiopia,USAGE,USG_TELEBIRR_USERS,increase,high,3
2,2021-05-17,product_launch,First major mobile money service in Ethiopia,USAGE,USG_P2P_COUNT,increase,high,6
3,2022-08-01,market_entry,End of state telecom monopoly,ACCESS,ACC_4G_COV,increase,medium,12
4,2022-08-01,market_entry,End of state telecom monopoly,AFFORDABILITY,AFF_DATA_INCOME,decrease,medium,12
5,2023-08-01,product_launch,Second mobile money entrant,USAGE,USG_MPESA_USERS,increase,high,3
6,2023-08-01,product_launch,Second mobile money entrant,ACCESS,ACC_MM_ACCOUNT,increase,medium,6
7,2024-01-01,infrastructure,National biometric digital ID system,ACCESS,ACC_OWNERSHIP,increase,medium,24
8,2024-01-01,infrastructure,National biometric digital ID system,GENDER,GEN_GAP_ACC,decrease,medium,24
9,2024-07-29,policy,Birr float introduced,AFFORDABILITY,AFF_DATA_INCOME,increase,high,3


In [21]:
# Save enriched datasets
with pd.ExcelWriter("../data/processed/ethiopia_fi_unified_data_enriched.xlsx") as writer:
    df_data.to_excel(writer, sheet_name="ethiopia_fi_unified_data", index=False)
    df_impact.to_excel(writer, sheet_name="Impact_sheet", index=False)


**Next Steps:**
- Task 2 will perform exploratory data analysis on the enriched dataset
- Visualize trends in Access and Usage
- Map events to indicators to estimate historical impact
